In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Your work should contain: data preparation steps (+ at least one plot), analysis of features, selection of regression algorithm,algorithm tuning (if required), costs prediction (*.csv or *.xlsx format)
# Your task is to predict costs for the missing month (december)

In [ ]:
dataframe = pd.read_csv('../input/costpredictions1/cost-predictions1.csv')
dataframe.head()

In [ ]:
dataset = dataframe.values
dataset

In [ ]:
dataframe = dataframe.rename(columns={' Delivered_cases': 'Delivered_cases'})

In [ ]:
agg_channel = pd.pivot_table(dataframe, index='Month', columns='Channel',
                                values='costs', aggfunc=np.mean).values
plt.figure(figsize=(14, 5))
plt.plot(agg_channel / agg_channel.mean(0)[np.newaxis])
plt.title("Channels")
plt.xlabel("Month")
plt.ylabel("Relative costs")

Надо проверить наличие пустых ячеек и заполнить их подходящей информацией.

In [ ]:
dataframe.info()
dataframe.isnull().sum()

In [ ]:
missing = [col for col in dataframe.columns
                     if dataframe[col].isnull().any()]
print("Missing values in:")
print(missing)

Заполняем численные показатели средними значениями.

In [ ]:
dataframe['Month'].fillna('8', inplace = True)
dataframe['km'].fillna(dataframe['km'].mean(),inplace=True)
dataframe['Delivered_cases'].fillna(dataframe['Delivered_cases'].mean(),inplace=True)
missing = [col for col in dataframe.columns
                     if dataframe[col].isnull().any()]
print("Missing values in:")
print(missing)

In [ ]:
plt.figure(figsize=(20,5))
dataframe['Customer_ID'].value_counts().head(100).plot.bar()
plt.show()

Заполняем категории самыми популярными значениями.

In [ ]:
dataframe['Customer_ID'] = dataframe['Customer_ID'].fillna('8473.0')
dataframe.isnull().sum()

In [ ]:
plt.figure(figsize=(20,5))
dataframe['Channel'].value_counts().head(100).plot.bar()
plt.show()

In [ ]:
dataframe['Channel'] = dataframe['Channel'].fillna('Food Store')
dataframe.isnull().sum()

Теперь остались пустыми только затраты, которые надо спрогнозировать

In [ ]:
sns.barplot(x = 'Month', y = 'costs', data = dataframe)

In [ ]:
dataframe = dataframe.drop(dataframe.columns[[0, 1, 2, 3]], axis=1)
dataframe.isnull().sum()

In [ ]:
sns.scatterplot(data=dataframe['costs'])

In [ ]:
dataframe['T'] = np.arange(len(dataframe))
print (dataframe)

In [ ]:
cols = dataframe.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataframe = dataframe[cols] 

In [ ]:
dataframe.plot.line(x='T', y=['Delivered_cases', 'km', 'costs'], xlim = [0, 5000], ylim = [0, 15000])

Перейдем к использованию модели для предсказания издержек.

In [ ]:
dataframe1 = dataframe[dataframe['Month']!=12]
dataframe1

In [ ]:
dataframe_test = dataframe[dataframe['Month']==12]
dataframe_test

In [ ]:
X = dataframe1.iloc[:,0:4].values
Y = dataframe1.iloc[:,4].values

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split (X, Y, test_size=0.2, random_state=5)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
def models (X_train,Y_train):
    from sklearn.ensemble import GradientBoostingRegressor
    xgbm = GradientBoostingRegressor(n_estimators=2000, 
                                      learning_rate =0.4, max_depth =3, random_state=5)
    xgbm.fit(X_train,Y_train)
    print ('XGB Training Accuracy:' , xgbm.score(X_train,Y_train))
    return xgbm

In [ ]:
model = models(X_train,Y_train)

In [ ]:
Xt = dataframe_test.iloc[:,0:4].values
Xt = sc.transform(Xt)

In [ ]:
dataframe_test['costs'] = model.predict(Xt)

In [ ]:
dataframe_test

In [ ]:
output = dataframe_test
output.to_csv('my_submission.csv', index=False)
